In [1]:
# from pyicecake import pyicecake
import numpy as np
import types
import collections
from random import shuffle
from nvidia.dali.pipeline import Pipeline
import nvidia.dali.ops as ops
import nvidia.dali.types as types
import PIL
from PIL import Image
import cupy as cp
from timeit import default_timer as timer
from matplotlib.pyplot import imshow
import cv2
import os
from tqdm.notebook import trange, tqdm
import time
%matplotlib inline

In [2]:
# rootDIR = "/home/lwangay/datasets/"
rootDIR= "/tmp/imageNet/"
file_list = "/home/lwangay/datasets/"+"UCF-101.txt"
filenames = []
lines = []
with open(file_list, 'r') as f:
    lines = f.readlines()
for l in lines:
    filenames.append(l.split(" ")[0].strip())

In [3]:
filenames[:10]

['UCF-101/CuttingInKitchen/v_CuttingInKitchen_g01_c01.avi',
 'UCF-101/CuttingInKitchen/v_CuttingInKitchen_g01_c02.avi',
 'UCF-101/CuttingInKitchen/v_CuttingInKitchen_g01_c03.avi',
 'UCF-101/CuttingInKitchen/v_CuttingInKitchen_g01_c04.avi',
 'UCF-101/CuttingInKitchen/v_CuttingInKitchen_g01_c05.avi',
 'UCF-101/CuttingInKitchen/v_CuttingInKitchen_g02_c01.avi',
 'UCF-101/CuttingInKitchen/v_CuttingInKitchen_g02_c02.avi',
 'UCF-101/CuttingInKitchen/v_CuttingInKitchen_g02_c03.avi',
 'UCF-101/CuttingInKitchen/v_CuttingInKitchen_g02_c04.avi',
 'UCF-101/CuttingInKitchen/v_CuttingInKitchen_g03_c01.avi']

In [4]:
# images = []
def extract_frames_skip(f,skip=1):
    fname = rootDIR+f
    video_file_size = []
    video_file_size.append(os.path.getsize(fname))

    frame_sizes = []
    vidcap = cv2.VideoCapture(fname)
    frame_count = int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT))
    print("total:{}".format(frame_count))
    for i in range(0, frame_count, skip):
        vidcap.set(cv2.CAP_PROP_POS_FRAMES,i)
        success,image = vidcap.read(1)
#         images.append(np.copy(image))
        cv2.imwrite("/tmp/out2.JPEG", image)
        frame_sizes.append(os.path.getsize("/tmp/out2.JPEG"))
#         time.sleep(5)
    vidcap.release()
    return np.asarray(video_file_size),np.asarray(frame_sizes)

In [5]:
# images = []
def extract_frames_random(f,frames = 15):
    fname = rootDIR+f
    video_file_size = []
    video_file_size.append(os.path.getsize(fname))
    frame_sizes = []
    vidcap = cv2.VideoCapture(fname)
    frame_count = int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT))
    print("total:{}".format(frame_count))
    random_frame = np.random.randint(0, frame_count - frames)
    print("rand start frame:{}".format(random_frame))
    vidcap.set(cv2.CAP_PROP_POS_FRAMES,random_frame)
    for i in range(0, frames):
        success,image = vidcap.read(1)
#         images.append(np.copy(image))
        cv2.imwrite("/tmp/out2.JPEG", image)
        frame_sizes.append(os.path.getsize("/tmp/out2.JPEG"))
#         time.sleep(5)
    vidcap.release()
    return np.asarray(video_file_size),np.asarray(frame_sizes)

In [29]:
print(filenames[0])
frame_sizes = []
video_file_size = []
for f in filenames[:100]:
    vs, fs = extract_frames_random(f, 15)
    frame_sizes.extend(fs)
    video_file_size.extend(vs)

UCF-101/CuttingInKitchen/v_CuttingInKitchen_g01_c01.avi
total:301
rand start frame:245
total:105
rand start frame:75
total:162
rand start frame:133
total:169
rand start frame:24
total:219
rand start frame:80
total:80
rand start frame:59
total:89
rand start frame:22
total:123
rand start frame:20
total:147
rand start frame:31
total:266
rand start frame:204
total:201
rand start frame:32
total:266
rand start frame:112
total:260
rand start frame:225
total:97
rand start frame:21
total:152
rand start frame:10
total:141
rand start frame:124
total:182
rand start frame:22
total:208
rand start frame:48
total:237
rand start frame:168
total:234
rand start frame:194
total:251
rand start frame:105
total:232
rand start frame:58
total:238
rand start frame:172
total:141
rand start frame:20
total:243
rand start frame:3
total:273
rand start frame:137
total:226
rand start frame:84
total:168
rand start frame:40
total:151
rand start frame:103
total:81
rand start frame:48
total:64
rand start frame:19
total:62

In [30]:
np.sum(frame_sizes)

30045031

In [31]:
np.sum(frame_sizes)/1024.

29340.8505859375

In [32]:
total_read = 75062338
total_frame_sizes = np.sum(frame_sizes)
total_video_file_size = np.sum(video_file_size)
print("random read 15 frames")

print("raw file size: {:.6f}KB".format(total_video_file_size/1024.))
print("read from disk: {:.6}KB".format(total_read/1024.))
print("frame_sizes: {:.6}KB".format(total_frame_sizes/1024.))
size = total_read if total_read < total_video_file_size else total_video_file_size
print("frame to disk read ratio: {:.6}%".format(100.0 * total_frame_sizes/size))

random read 15 frames
raw file size: 52221.509766KB
read from disk: 73303.1KB
frame_sizes: 29340.9KB
frame to disk read ratio: 56.1854%


In [33]:
print(filenames[0])
frame_sizes = []
video_file_size = []
for f in filenames[:100]:
    vs,fs = extract_frames_skip(f, 15)
    frame_sizes.extend(fs)
    video_file_size.extend(vs)
    

UCF-101/CuttingInKitchen/v_CuttingInKitchen_g01_c01.avi
total:301
total:105
total:162
total:169
total:219
total:80
total:89
total:123
total:147
total:266
total:201
total:266
total:260
total:97
total:152
total:141
total:182
total:208
total:237
total:234
total:251
total:232
total:238
total:141
total:243
total:273
total:226
total:168
total:151
total:81
total:64
total:62
total:98
total:190
total:193
total:253
total:248
total:209
total:122
total:158
total:131
total:319
total:310
total:328
total:285
total:351
total:155
total:394
total:119
total:128
total:70
total:54
total:174
total:110
total:149
total:116
total:148
total:138
total:146
total:138
total:100
total:82
total:104
total:98
total:118
total:155
total:100
total:99
total:176
total:198
total:206
total:196
total:185
total:180
total:243
total:140
total:97
total:173
total:82
total:156
total:58
total:49
total:50
total:53
total:289
total:277
total:251
total:276
total:209
total:173
total:170
total:175
total:150
total:177
total:137
total:261
to

In [34]:
print(len(video_file_size))

100


In [35]:
total_read = 560891808
total_frame_sizes = np.sum(frame_sizes)
total_video_file_size = np.sum(video_file_size)
print("raw file size: {:.6f}KB".format(total_video_file_size/1024.))
print("read one frame every 15 frames")
print("read from disk: {:.6f}KB".format(total_read/1024.))
print("frame_sizes: {:.6}KB".format(total_frame_sizes/1024.))
size = total_read if total_read < total_video_file_size else total_video_file_size
print("frame to disk read ratio: {:.6}%".format(100.0 * total_frame_sizes/size))

raw file size: 52221.509766KB
read one frame every 15 frames
read from disk: 547745.906250KB
frame_sizes: 22296.3KB
frame to disk read ratio: 42.6957%
